# Introduction to Python Programming

## Walkthrough: Exercise 4.9 - Plotting with Bokeh

First things first, we need to import the functions that we need from the third-party libraries, Pandas and Bokeh. We import all of Pandas, with the abbreviated namespace `pd`, and `figure`, `show` and `output_notebook` from `bokeh.plotting`. We use `output_notebook` here, to allow the plot to be displayed in this Jupyter Notebook - you might want to use `output_file` instead, which will result in the figure being rendered in an HTML file instead (see below). `figure` is used to construct the figure, and `show` is used to render the layout in the end. Speaking of layout, we import `gridplot` from `bokeh.io`, to create rows and columsn of the figures after we've created them. 

In [7]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import gridplot

Now, we need to read the data from the file into Python. Unlike in the matplotlib example, we will use a Pandas `DataFrame` here. (Note that this approach would work with matplotlib too - we're just choosing to introduce it at the same time as we start doing things with Bokeh.) We use the `read_table` function, with the `index_col` and `header` arguments, to read the tabular data into a dataframe.

In [8]:
inputFile = 'speciesDistribution_tabular.txt'
data = pd.read_table(inputFile, header=0, index_col=0)
data

,Grimston Wood,Hagg Wood,Hetchell Wood N,Hetchell Wood S,Scoreby Wood,Sutton Wood,Wheldrake Wood
taxonID,,,,,,,
A,123,2039,12983,9380,920,883,0
B,1340,9394,8493,13928,3928,293,91
C,11984,19380,948,0,0,893,22649
D,0,9102,9384,949,9301,18990,2949
E,9389,932,4942,19023,19384,0,901
F,4320,0,0,9384,12949,3910,0
G,1283,893,9834,948,0,930,9204
H,0,5839,0,9284,3892,1738,2040
I,0,0,1293,0,9192,819,8173


Great! Now we have all of our data loaded and arranged in the way that we need, we can move onto plotting.

Now we use `output_notebook()` to make sure that our plots can be displayed in this Notebook.

In [9]:
output_notebook()

Loading BokehJS ...

_Note: to save the plot to a file instead, use:_

```Python
output_file('my_amazing_plot.html') # give files descriptive names so you can easily identify them in future
```

Now we can start creating the indivual bar plots, one for each site. We want to display these plots all together, so we should start by storing them in a list. For each site we pull out the bar heights - this is the dataframe column corresponding to the site name; and the taxon names as a list created from the index column of the dataframe.

Next, we create a figure object for the site, using the site name as the title, and setting the range for the x-axis as the taxon names. `tools=[]` disables the interactive tool bar for these plots. Finally, we call the `figure` object's `vbar` method to add the bars to the figure axes. The mid-points of the bars on the axis need to be labelled with the taxon names, so we construct a list comprehension* adding 0.5 to the index numbers along the axis. The heights of the bars are given as the column that we pulled out earlier. Once the figure is constructed, we add it to the list of plots.

In [10]:
plots = []
for site in data.columns:
    heights = data[site]
    taxa = list(data.index)
    fig = figure(plot_width=300, plot_height=200, title=site, x_range=taxa, tools=[])
    fig.vbar(x=[n+0.5 for n in range(13)], 
       width=0.8, 
       bottom=0, 
       top=heights, 
       color='firebrick')
    plots.append(fig)

The final step is to lay out and display the plots. Here, we will use the `gridplot` function from `bokeh.io`, but there are plenty of other options. Check out the documentation [here](http://bokeh.pydata.org/en/latest/docs/user_guide/layout.html).

In [11]:
layout = gridplot([[plots[0],plots[1]],\
                   [plots[2],plots[3]],\
                   [plots[4],plots[5]],\
                   [plots[6]]])
show(layout)

As a bonus, let's colour the bars individually according to the taxon that they refer to. There are 13 taxa, which is a bit too large to avoid using some similar colours, so we use the Category20 color palette instead. Several palettes are available via the `bokeh.palettes` module. Choose which one you want, then specify how many colours you need when you use the palette in your plot(s).

In [12]:
from bokeh.palettes import Category20

In [13]:
plots = []
for site in data.columns:
    heights = data[site]
    taxa = list(data.index)
    fig = figure(plot_width=300, plot_height=200, title=site, x_range=taxa, tools=[])
    fig.vbar(x=[n+0.5 for n in range(13)], 
       width=0.8, 
       bottom=0, 
       top=heights, 
       color=Category20[13])
    plots.append(fig)

In [20]:
layout = gridplot([[plots[0],plots[1]],\
                   [plots[2],plots[3]],\
                   [plots[4],plots[5]],\
                   [plots[6]]])
show(layout)

#### \*Note on list comprehensions

A list comprehension is a way of building up a list from a loop on the fly. For example, the following two code blocks are equivalent:

In [15]:
originals = range(10)
squares = []
for num in originals:
    squares.append(num**2)
print(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [16]:
squares = [ x**2 for x in range(10) ]
print(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


The list comprehension was necessary above, to align the x-axis labels correctly with the centre of each taxon bar. Another approach to get the same result is possible using the `arange` function from the `numpy` library, which provides an _array_ of numbers in the range requested. Consider the difference in output between the two blocks below:

In [17]:
some_numbers = list(range(2, 20)) # a standard list object
print(some_numbers)
some_numbers + 0.5

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


TypeError: can only concatenate list (not "float") to list

In [18]:
from numpy import arange
some_numbers = arange(2, 20)
some_numbers + 0.5

array([  2.5,   3.5,   4.5,   5.5,   6.5,   7.5,   8.5,   9.5,  10.5,
        11.5,  12.5,  13.5,  14.5,  15.5,  16.5,  17.5,  18.5,  19.5])

So, if we wanted to, we could use `arange` to create our plots as follows:

In [21]:
plots = []
for site in data.columns:
    heights = data[site]
    taxa = list(data.index)
    fig = figure(plot_width=300, plot_height=200, title=site, x_range=taxa, tools=[])
    fig.vbar(x=arange(13)+0.5, 
       width=0.8, 
       bottom=0, 
       top=heights, 
       color=Category20[13])
    plots.append(fig)

In [23]:
layout = gridplot([[plots[0],plots[1]],\
                   [plots[2],plots[3]],\
                   [plots[4],plots[5]],\
                   [plots[6]]])
show(layout)